In [ ]:
!pip install -Uqq fastai
!pip install -Uqq transformers datasets

     |████████████████████████████████| 4.9 MB 13.9 MB/s 
     |████████████████████████████████| 431 kB 66.5 MB/s 
     |████████████████████████████████| 120 kB 71.8 MB/s 
     |████████████████████████████████| 6.6 MB 45.1 MB/s 
     |████████████████████████████████| 212 kB 74.2 MB/s 
     |████████████████████████████████| 115 kB 68.6 MB/s 
     |████████████████████████████████| 127 kB 65.5 MB/s 


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset

In [ ]:
model_name = 'distilbert-base-uncased'

In [ ]:
tokz = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def preprocess(example):
  return tokz(example['text'], truncation=True)

In [ ]:
imdb = load_dataset("imdb")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/2fdd8b9bcadd6e7055e742a706876ba43f19faee861df134affd7a3f60fc38a1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
small_train_dataset = imdb["train"].shuffle(seed=42).select([i for i in list(range(3000))])
small_test_dataset = imdb["test"].shuffle(seed=42).select([i for i in list(range(300))])

In [ ]:
tokenized_train = small_train_dataset.map(preprocess, batched=True)
tokenized_test = small_test_dataset.map(preprocess, batched=True)

  0%|          | 0/3 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokz)

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.bias', 'classifier

In [ ]:
!pip install -Uqq evaluate

     |████████████████████████████████| 69 kB 301 kB/s 


In [ ]:
import evaluate

In [ ]:
import numpy as np
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  metrics = evaluate.load("glue", "mrpc")
  predictions = np.argmax(logits, axis=-1)
  return metrics.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

In [ ]:
training_args = TrainingArguments(output_dir='results',
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=32,
                                  num_train_epochs=5,
                                  weight_decay=0.1,
                                  save_strategy="epoch",
                                  evaluation_strategy="epoch")

trainer = Trainer(model = model,
                  args = training_args,
                  train_dataset = tokenized_train,
                  eval_dataset = tokenized_test,
                  tokenizer=tokz,
                  data_collator=data_collator,
                  compute_metrics=compute_metrics)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 3000
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 940


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.465056,0.876667,0.875421
2,No log,0.470814,0.876667,0.873720
3,0.132100,0.644269,0.866667,0.872611
4,0.132100,0.665001,0.870000,0.872964
5,0.132100,0.681259,0.866667,0.871795


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size = 32
Saving model checkpoint to results/checkpoint-188
Configuration saved in results/checkpoint-188/config.json
Model weights saved in results/checkpoint-188/pytorch_model.bin
tokenizer config file saved in results/checkpoint-188/tokenizer_config.json
Special tokens file saved in results/checkpoint-188/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 300
  Batch size =

TrainOutput(global_step=940, training_loss=0.08494734764099121, metrics={'train_runtime': 257.568, 'train_samples_per_second': 58.237, 'train_steps_per_second': 3.65, 'total_flos': 1964686083560256.0, 'train_loss': 0.08494734764099121, 'epoch': 5.0})

In [ ]:
input = "I like this movie!"
tokenized = tokz(input, return_tensors='pt').to('cuda')
tokenized

{'input_ids': tensor([[ 101, 1045, 2066, 2023, 3185,  999,  102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [ ]:
import torch
with torch.no_grad():
  output = model(**tokenized)


In [ ]:
logits = output[0].detach().cpu().numpy()
predictions = np.argmax(logits, axis=-1)
if predictions[0] == 1:
  print("Positive with probability: ", logits[0])
else:
  print("Negative with probability: ", logits[0])

Positive with probability:  [-3.0709689  3.1089528]
